In [1]:
#imports 
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [15]:
df = pd.read_csv('./data/newsCorpora.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
df = df[['TITLE','CATEGORY']]
df.head()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [16]:
my_dict = {
    'e':'Entertainment',
    'b':'Business',
    't':'Science',
    'm':'Health'
}

def update_cat(x):
    return my_dict[x]

df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

df.head()


,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",Business,0
1,Fed's Charles Plosser sees high bar for change...,Business,0
2,US open: Stocks fall after Fed official hints ...,Business,0
3,"Fed risks falling 'behind the curve', Charles ...",Business,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business,0


In [17]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [28]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [33]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

{'sequence': 'I am a very hard worker so let me take care of everything for you',
 'labels': ['bee', 'leech', 'ant'],
 'scores': [0.46051937341690063, 0.29015854001045227, 0.2493220567703247]}

In [35]:
classifier(
    "Do this work for me.",
    candidate_labels=["bee", "ant", "leech"],
)

{'sequence': 'Do this work for me.',
 'labels': ['bee', 'leech', 'ant'],
 'scores': [0.42481550574302673, 0.37177881598472595, 0.20340566337108612]}